In [47]:
import os
import sys
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt

%matplotlib inline

# Hack to ensure the notebook can load local modules by appending the parent directory to the path
# Ensure a '.env' file is in the workspace root
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))
from alphasim.backtest import backtest

In [48]:
price_df = pdr.get_data_yahoo(['VTI', 'TLT'])
price_df = price_df['Adj Close']

display(price_df)

Symbols,VTI,TLT
Date,,
2017-10-09,120.264015,111.911888
2017-10-10,120.548782,112.092400
2017-10-11,120.714119,112.309013
2017-10-12,120.567154,112.787460
2017-10-13,120.658997,113.599838
...,...,...
2022-09-30,179.470001,102.206001
2022-10-03,184.029999,103.830002
2022-10-04,189.940002,103.540001


In [49]:
weight_df = price_df.copy()
weight_df['VTI'] = 0.6
weight_df['TLT'] = 0.4

display(weight_df)

Symbols,VTI,TLT
Date,,
2017-10-09,0.6,0.4
2017-10-10,0.6,0.4
2017-10-11,0.6,0.4
2017-10-12,0.6,0.4
2017-10-13,0.6,0.4
...,...,...
2022-09-30,0.6,0.4
2022-10-03,0.6,0.4
2022-10-04,0.6,0.4


In [50]:
def min_commission(trade_price, trade_size):
    return 10

In [51]:
max_lev = 1
vola_target = 0.10
trade_buffer = 0.05
initial_cap = 10000.0
result = backtest(price_df, weight_df, min_commission, trade_buffer, initial_cap)

In [52]:
assert(price_df.shape == weight_df.shape)

price_df['cash'] = 1
weight_df['cash'] = max_lev - weight_df.sum(axis=1)

pf_df = pd.DataFrame(index=price_df.index, columns=weight_df.columns)
pf_df[:] = 0.0

pnl_df = pf_df.copy()


'''
current_weight = (size * price) / nav
target_weight = weight
delta_weight = target_weight - current_weight
do_trade = delta_weight > trade_buffer

In fixed_pct_comm scheme:
trade_weight = target_weight - trade_buffer

In min_comm scheme:
trade_weight = target_weight

trade_value = trade_weight * nav
trade_size = trade_value / price
'''

display(pf_df)

Symbols,VTI,TLT,cash
Date,,,
2017-10-09,0.0,0.0,0.0
2017-10-10,0.0,0.0,0.0
2017-10-11,0.0,0.0,0.0
2017-10-12,0.0,0.0,0.0
2017-10-13,0.0,0.0,0.0
...,...,...,...
2022-09-30,0.0,0.0,0.0
2022-10-03,0.0,0.0,0.0
2022-10-04,0.0,0.0,0.0


In [53]:
n = len(price_df)

result = pd.DataFrame()

for i in range(n):

    start_pf = None
    if i == 0:
        start_pf = pf_df.iloc[0].copy()
        start_pf['cash'] = initial_cap
    else:
        start_pf = pf_df.iloc[i-1].copy()

    # Slice data to next time interval (t)
    t = pf_df.index[i]
    price = price_df.iloc[i]
    pnl = pnl_df.iloc[i]

    # Mark-to-market the portfolio 
    pnl = start_pf * price
    nav = pnl.sum()

    if nav <= 0:
        break

    # Calc latest portfolio weights based on NAV
    curr_weight = pnl / nav
     
    # Calc delta of current to target weight
    target_weight = weight_df.iloc[i].copy()
    delta_weight = target_weight - curr_weight

    # Based on buffer decide if trade should be made
    do_trade = delta_weight > trade_buffer
    do_trade['cash'] = False

    # If no trade indicated then set target weight to current weight
    target_weight.loc[~do_trade] = curr_weight

    # Assume min fixed commission so trade to target weight
    adj_target_weight = target_weight
    adj_delta_weight = adj_target_weight - curr_weight

    # Calc trade to achieve adjusted target weight
    trade_value = adj_delta_weight * nav
    trade_size = trade_value / price

    # Calc commission
    #fee = min_commission(price, trade_size)

    end_pf = start_pf.copy()
    end_pf = end_pf + trade_size
    #post_trade_pf['cash'] -= fee
    #post_trade_pf['cash'] -= trade_value.sum()
    #post_trade_exposure = post_trade_pf * price

    #adj_target_value = adj_target_weight * nav
    #adj_target_size = adj_target_value / price
    pf_df.iloc[i] = end_pf
    

    # Append data for this time interval to the result 
    series = pd.concat(
        [price, start_pf, pnl, curr_weight, target_weight, delta_weight, do_trade, 
            adj_target_weight, adj_delta_weight, trade_value, trade_size, end_pf], 
        keys=['price', 'start_pf', 'pnl', 'curr_weight', 'target_weight', 'delta_weight', 'do_trade', 
            'adj_target_weight', 'adj_delta_weight', 'trade_value', 'trade_size', 
            'end_pf'],
        axis=1)
    series['datetime'] = t
    series = series.set_index(['datetime', series.index])
    result = pd.concat([result, series])


display(result.head(10*3))


price    start_pf           pnl curr_weight  \
datetime   Symbols                                                     
2017-10-09 VTI      120.264015         0.0           0.0         0.0   
           TLT      111.911888         0.0           0.0         0.0   
           cash       1.000000     10000.0       10000.0         1.0   
2017-10-10 VTI      120.548782   49.890235     6014.2071      0.3004   
           TLT      112.092400   35.742405   4006.451914    0.200116   
           cash       1.000000     10000.0       10000.0    0.499484   
2017-10-11 VTI      120.714119   99.647588  12028.870803     0.40026   
           TLT      112.309013   71.443413   8023.739233     0.26699   
           cash       1.000000     10000.0       10000.0     0.33275   
2017-10-12 VTI      120.567154  149.374126  18009.613249    0.449321   
           TLT      112.787460  107.035434  12072.254792     0.30119   
           cash       1.000000     10000.0       10000.0    0.249489   
2017-10-13 VTI      120.658997  199.466605  24067.440367    0.479282   
           TLT      113.599838  142.150086  16148.226809    0.321577   
           cash       1.000000     10000.0       10000.0    0.199141   
2017-10-16 VTI      120.741646  249.707035    30150.0384    0.500649   
           TLT      113.518578  176.815981  20071.898683    0.333299   
           cash       1.000000     10000.0       10000.0    0.166052   
2017-10-17 VTI      120.796768  299.260143   36149.65818    0.514446   
           TLT      113.663055  212.201169  24119.433201    0.343244   
           cash       1.000000     10000.0       10000.0     0.14231   
2017-10-18 VTI      120.952942  349.028003  42215.963825    0.526789   
           TLT      112.913803  247.289116  27922.354608    0.348427   
           cash       1.000000     10000.0       10000.0    0.124784   
2017-10-19 VTI      120.980515  397.534697  48093.952138    0.533159   
           TLT      113.112473  283.892018   32111.72804    0.355983   
           cash       1.000000     10000.0       10000.0    0.110858   
2017-10-20 VTI      121.632637  447.372937  54415.150064    0.565728   
           TLT      111.911888  283.892018  31770.891709    0.330307   
           cash       1.000000     10000.0       10000.0    0.103965   

                   target_weight delta_weight  do_trade adj_target_weight  \
datetime   Symbols                                                          
2017-10-09 VTI               0.6          0.6      True               0.6   
           TLT               0.4          0.4      True               0.4   
           cash              1.0         -1.0     False               1.0   
2017-10-10 VTI               0.6       0.2996      True               0.6   
           TLT               0.4     0.199884      True               0.4   
           cash         0.499484    -0.499484     False          0.499484   
2017-10-11 VTI               0.6      0.19974      True               0.6   
           TLT               0.4      0.13301      True               0.4   
           cash          0.33275     -0.33275     False           0.33275   
2017-10-12 VTI               0.6     0.150679      True               0.6   
           TLT               0.4      0.09881      True               0.4   
           cash         0.249489    -0.249489     False          0.249489   
2017-10-13 VTI               0.6     0.120718      True               0.6   
           TLT               0.4     0.078423      True               0.4   
           cash         0.199141    -0.199141     False          0.199141   
2017-10-16 VTI               0.6     0.099351      True               0.6   
           TLT               0.4     0.066701      True               0.4   
           cash         0.166052    -0.166052     False          0.166052   
2017-10-17 VTI               0.6     0.085554      True               0.6   
           TLT               0.4     0.056756      True               0.4   
           cash          0

In [54]:
#trades = result.loc[result['do_trade'] == True]

#display(trades)
#display(trades.describe())

In [55]:
#result = result.iloc[2:,:]

display(result)


ret_df = np.log(result['pnl']/result['pnl'].shift(1)).dropna()
cumret_df = ret_df.cumsum()

ax = cumret_df.plot(figsize=(15, 6))
ax.grid(True)
plt.title('Cum. Returns')
plt.legend(loc=2)
plt.show()

price     start_pf            pnl curr_weight  \
datetime   Symbols                                                       
2017-10-09 VTI      120.264015          0.0            0.0         0.0   
           TLT      111.911888          0.0            0.0         0.0   
           cash       1.000000      10000.0        10000.0         1.0   
2017-10-10 VTI      120.548782    49.890235      6014.2071      0.3004   
           TLT      112.092400    35.742405    4006.451914    0.200116   
...                        ...          ...            ...         ...   
2022-10-05 TLT      102.550003  1543.274368   158262.79118    0.358402   
           cash       1.000000      10000.0        10000.0    0.022646   
2022-10-06 VTI      187.779999  1441.694969  270721.479485    0.617873   
           TLT      102.010002  1543.274368  157429.421608    0.359304   
           cash       1.000000      10000.0        10000.0    0.022823   

                   target_weight delta_weight  do_trade adj_target_weight  \
datetime   Symbols                                                          
2017-10-09 VTI               0.6          0.6      True               0.6   
           TLT               0.4          0.4      True               0.4   
           cash              1.0         -1.0     False               1.0   
2017-10-10 VTI               0.6       0.2996      True               0.6   
           TLT               0.4     0.199884      True               0.4   
...                          ...          ...       ...               ...   
2022-10-05 TLT          0.358402     0.041598     False          0.358402   
           cash         0.022646    -0.022646     False          0.022646   
2022-10-06 VTI          0.617873    -0.017873     False          0.617873   
           TLT          0.359304     0.040696     False          0.359304   
           cash         0.022823    -0.022823     False          0.022823   

                   adj_delta_weight  trade_value trade_size       end_pf  
datetime   Symbols                                                        
2017-10-09 VTI                  0.6       6000.0  49.890235    49.890235  
           TLT                  0.4       4000.0  35.742405    35.742405  
           cash                 0.0          0.0        0.0      10000.0  
2017-10-10 VTI               0.2996  5998.188308  49.757353    99.647588  
           TLT             0.199884  4001.811692  35.701008    71.443413  
...                             ...          ...        ...          ...  
2022-10-05 TLT                  0.0          0.0        0.0  1543.274368  
           cash                 0.0          0.0        0.0      10000.0  
2022-10-06 VTI                  0.0          0.0        0.0  1441.694969  
           TLT                  0.0          0.0        0.0  1543.274368  
           cash                 0.0          0.0        0.0      10000.0  

[3774 rows x 12 columns]

ZeroDivisionError: float division by zero